In [ ]:
import boto3
from torch.utils.data import DataLoader
import torch
import os
import json
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from sache import SpecifiedHookedViT
from vitact.filedataset import FilePathDataset
from vitact.download import download_laion
from vitact.top9 import download_sae_checkpoints

In [ ]:
locations = [(2, "resid")]
transformer_name = "laion/CLIP-ViT-L-14-laion2B-s32B-b82K"
device = "cuda"
transformer = SpecifiedHookedViT(locations, transformer_name, device=device)


In [ ]:
sae_checkpoints = [
        's3://sae-activations/log/CLIP-ViT-L-14/2_resid/2_resid_29c579/600023040.pt',
]
sae_paths = download_sae_checkpoints(sae_checkpoints, base_dir='cruft')

In [ ]:
sae = torch.load(sae_paths[0], map_location=device)

In [ ]:

# Prepare locations and load SAEs
    
    sae_dict[location] = sae

if not os.path.exists(image_dir):
    os.makedirs(image_dir)

num_features_dict = {}
topk_values_dict = {}
topk_indices_dict = {}
cumulative_file_paths = []
cumulative_index = 0

with torch.no_grad():
    for i, (paths, batch) in tqdm(enumerate(dataloader), total=n_activations // batch_size):
        activations = transformer.all_activations(batch)
        current_batch_size = batch.size(0)  # Avoid shadowing the outer batch_size
        current_indices = torch.arange(cumulative_index, cumulative_index + current_batch_size, dtype=torch.long, device=device)
        cumulative_index += current_batch_size

        cumulative_file_paths.extend(paths)

        for location in locations:
            layer, hook_name = location
            sae = sae_dict[location]
            activation = activations[location]
            activation = activation[:, 1]

            latent = sae.forward_descriptive(activation)['latent']
            latent = latent.detach()